# Phase 5: Evaluation and Comparison

**Objective**: Compare baseline models (Phase 3) and mitigated models (Phase 4) to assess the effectiveness of imbalance mitigation strategies (SMOTE, Random Undersampling, NearMiss, Weighted Loss) for 3-class (Negative, Neutral, Positive) sentiment classification on the Bangla Sentiment Dataset. Evaluate performance on the test set, analyze source-specific performance (newspapers, social media, blogs) to test hypothesis H3 (source-specific differences in sentiment classification), and perform statistical tests to determine significant improvements.


### Step 1: Load Test Data and Models

- **Objective**: Load test TF-IDF matrix, labels, source metadata, and all models (baseline and mitigated)

In [3]:
import pandas as pd
import scipy.sparse as sp
import joblib
import os
import logging
from tqdm import tqdm

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define paths
data_dir = "text_representation/"

files = {
    'tfidf_test': f"{data_dir}tfidf_test.npz",
    'labels_test': f"{data_dir}labels_test.csv",
}

# Check file existence
for name, path in files.items():
    if not os.path.exists(path):
        logging.error(f"Missing file: {path}")
        raise FileNotFoundError(f"Missing file: {path}")

# Load test data
tfidf_test = sp.load_npz(files['tfidf_test'])
y_test = pd.read_csv(files['labels_test'], encoding='utf-8')['Label'].values

logging.info("Test data loaded successfully")

# Validate shapes
assert tfidf_test.shape[0] == len(y_test), "Test data mismatch"
logging.info("Data shapes validated")

2025-06-25 10:56:14,740 - INFO - Test data loaded successfully
2025-06-25 10:56:14,742 - INFO - Data shapes validated


In [4]:
# Load models
model_dir_baseline = "models/baseline_models/"
model_dir_mitigated = "models/mitigated_models/"

model_configs = [
    ('baseline', model_dir_baseline, ['LogisticRegression_tuned_grid', 'SVM_tuned_grid', 'NaiveBayes_tuned_grid', 'RandomForest_tuned_grid']),
    ('mitigated', model_dir_mitigated, [
        f"{model}_{mitigation}_tuned"
        for model in ['LogisticRegression', 'SVM', 'NaiveBayes', 'RandomForest']
        for mitigation in ['smote', 'undersampled', 'nearmiss', 'weighted']
    ])
]
models = {}
for config_type, model_dir, model_names in model_configs:
    for name in tqdm(model_names, desc=f"Loading {config_type} models"):
        try:
            models[f"{config_type}_{name}"] = joblib.load(f"{model_dir}{name}.joblib")
            logging.info(f"Loaded model: {config_type}_{name}")
        except Exception as e:
            logging.error(f"Error loading {name}: {str(e)}")

Loading baseline models:   0%|          | 0/4 [00:00<?, ?it/s]/home/fahad/projects/personal/sentiment_analysis_bangla/venv/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
2025-06-25 10:58:40,317 - INFO - Loaded model: baseline_LogisticRegression_tuned_grid
Loading baseline models:  25%|██▌       | 1/4 [00:01<00:05,  1.78s/it]/home/fahad/projects/personal/sentiment_analysis_bangla/venv/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scik